<a href="https://colab.research.google.com/github/Shuaib11-Github/AI_Agents/blob/main/Agentic_RAG_for_Analyzing_Customer_Issues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Agentic RAG for Analyzing Customer Issues: A Comprehensive Tutorial**

Retrieval-Augmented Generation (RAG) is a groundbreaking AI technique that boosts the performance of Large Language Models (LLMs) by integrating real-time document retrieval during text generation. Traditional RAG methods, such as Naive RAG, can falter when faced with complex, multi-step reasoning tasks. Enter **Agentic RAG**, which combines RAG's retrieval capabilities with autonomous decision-making agents, offering unparalleled performance in handling intricate queries.

This tutorial provides a hands-on guide to building an Agentic RAG system for analyzing customer issues, using Python and key libraries like **CrewAI**, **LangChain**, **OpenAI**, **Gradio**, and **ChromaDB**.

---

## **Learning Objectives**
- Understand the key differences between Agentic RAG and Naive RAG.
- Explore the limitations of Naive RAG in handling complex queries.
- Discover real-world use cases where Agentic RAG excels.
- Implement Agentic RAG in Python using CrewAI.
- Strengthen Naive RAG capabilities by adding decision-making agents.

---

## **Agentic RAG: Strengthening Naive RAG**
Agentic RAG enhances Naive RAG by adding autonomous decision-making agents capable of iterative reasoning. In Naive RAG, the retriever is passive, fetching information only when prompted. In contrast, Agentic RAG integrates **agents** that actively decide:
- **When** to retrieve information.
- **What** information to retrieve.
- **How** to synthesize the retrieved data for actionable insights.

### **Key Limitations of Naive RAG**
Naive RAG struggles with:
- **Summarization Questions**: E.g., "Summarize this document."
- **Comparative Analysis**: E.g., "Compare PepsiCo's and Coca-Cola's Q4 2023 strategies."
- **Complex Multi-Part Queries**: E.g., "Analyze retail inflation arguments in articles from *The Mint* and *Economic Times*, compare findings, and provide conclusions."

Agentic RAG addresses these challenges by incorporating reasoning agents capable of handling such queries effectively.

---

## **Use Cases of Agentic RAG**
1. **Legal Research**: Compare legal documents to extract key clauses.
2. **Market Analysis**: Analyze and compare top brands in a product segment.
3. **Medical Diagnosis**: Combine patient data with research for diagnoses.
4. **Financial Analysis**: Summarize and compare financial reports.
5. **Regulatory Compliance**: Ensure adherence by comparing policies with laws.

---

## **Building Agentic RAG with Python and CrewAI**

### **Step 1: Install Necessary Python Libraries**
Install the required Python libraries to set up the environment.




In [ ]:
!pip install llama-index-core
!pip install llama-index-readers-file
!pip install llama-index-embeddings-openai
!pip install llama-index-llms-llama-api
!pip install 'crewai[tools]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### **Step 2: Import Required Libraries**
Load essential libraries to set up agents, tools, and data processing.

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import LlamaIndexTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI

### **Step 3: Read Customer Issues Data**
Load customer issues data from a CSV file.

In [ ]:
reader = SimpleDirectoryReader(input_files=["/content/CustomerSuppTicket_small.csv"])
docs = reader.load_data()

### **Step 4: Define OpenAI API Key**
Set up the OpenAI API key to enable access to GPT-based models.

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('openai') # Replace with your OpenAI API key
os.environ['OPENAI_API_KEY'] = openai_api_key

### **Step 5: LLM Initialization**
Initialize the GPT-4 model for query processing.

In [ ]:
llm = OpenAI(model="gpt-4o")

### **Step 6: Create Vector Store Index and Query Engine**
Create a searchable index and query engine for retrieving similar documents.

In [ ]:
index = VectorStoreIndex.from_documents(docs)
query_engine = index.as_query_engine(similarity_top_k=5, llm=llm)

### **Step 7: Create a Query Tool**
Build a tool using the query engine for retrieving customer issue data.

In [ ]:
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Customer Support Query Tool",
    description="Use this tool to lookup the customer ticket data"
)

### **Step 8: Define Agents**
Define agents with distinct roles and goals.

In [ ]:
researcher = Agent(
    role="Customer Ticket Analyst",
    goal="Uncover insights about customer issue trends.",
    backstory="Analyze customer issues for brands like GoPro, LG, Dell, etc.",
    verbose=True,
    tools=[query_tool]
)

writer = Agent(
    role="Product Content Specialist",
    goal="Craft compelling content on customer issue trends.",
    backstory="Transform customer data into engaging narratives.",
    verbose=True
)

### **Step 9: Create Tasks**
Assign tasks to the agents.

In [ ]:
task1 = Task(
    description="Analyze customer issues for brands like GoPro, LG, Dell, etc.",
    expected_output="Detailed report with trends and insights.",
    agent=researcher
)

task2 = Task(
    description="Develop an engaging blog post summarizing customer issues for each brand.",
    expected_output="Bullet-point blog post highlighting key issues for each brand.",
    agent=writer
)

### **Step 10: Instantiate the Crew**
Combine agents and tasks into a crew and execute the process.

In [ ]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=True
)

result = crew.kickoff()

# Agent: Customer Ticket Analyst
## Task: Analyze customer issues for brands like GoPro, LG, Dell, etc.
 

I encountered an error while trying to use the tool. This was the error: 1 validation error for QueryToolSchema
query
  Field required [type=missing, input_value={'description': 'GoPro customer issues'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing.
 Tool Customer Support Query Tool accepts these inputs: Tool Name: Customer Support Query Tool
Tool Arguments: {'query': {'description': 'Search query for the query tool.', 'type': 'str'}}
Tool Description: Use this tool to lookup the customer ticket data



# Agent: Customer Ticket Analyst
## Thought: I need to gather data on customer issues for brands like GoPro, LG, and Dell to analyze trends and insights. I will start by querying customer support ticket data for these brands.
## Using tool: Customer Support Query Tool
## Tool Input: 
"{\"description\": \"GoPro customer issues\"}"
## T

---

### **Output**
The system generates a detailed analysis of customer issues and an engaging blog post summarizing key findings for each brand. For instance:

- **GoPro**: Common complaints include poor battery life and app connectivity.
- **Fitbit**: Issues with syncing and inaccurate step tracking.

---

## **Conclusion**
Agentic RAG marks a significant evolution in Retrieval-Augmented Generation by integrating decision-making agents. It enhances RAG’s ability to handle complex queries, providing robust insights across various industries. Using Python and CrewAI, developers can create intelligent systems for smarter, data-driven decisions.

---

## **Key Takeaways**
- Agentic RAG integrates autonomous agents for dynamic decision-making.
- It outperforms Naive RAG in handling multi-step reasoning and complex queries.
- Python and CrewAI simplify the implementation of Agentic RAG systems.

---






